In [23]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [9]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')
bucket_obj = bucket.objects.filter(Prefix='2022-01-28/')
objects = [obj for obj in bucket_obj]

In [ ]:
# 3. 

arg_date = '2021-05-07'
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)


# 1.

-- filter on date

bucket_obj1 = bucket.objects.filter(prefix='2021-03-15')
bucket_obj2 = bucket.objects.filter(prefix='2021-03-16')
objects = [obj for obj in bucket_obj11] + [obj for obj in bucket_obj21



--put this in its own cell to set the columns
csv_obj_init = bucket.Object(key=obj.key[0]).get().get('Body').read().decode('utf-8)
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')


df_init.columns



df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

In [10]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-28/2022-01-28_BINS_XETR10.csv'),
 s3.Object

read a csv file to view the contents

In [12]:
csv_obj = bucket.Object(key='2022-01-28/2022-01-28_BINS_XETR16.csv').get().get('Body').read().decode('utf-8')

In [18]:
data = StringIO(csv_obj)
df = pd.read_csv(data, delimiter=',')
df

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000ETFL011,EL4A,DK DAX,ETF,EUR,2504258,2022-01-28,16:00,137.24,137.24,137.24,137.24,13,1
1,DE000ETFL060,EL4F,DK DAX (AUSSCHUETTEND),ETF,EUR,2504259,2022-01-28,16:00,64.63,64.63,64.63,64.63,49,1
2,DE0002635273,EXSB,ISHARES DIVDAX UCITS ETF,ETF,EUR,2504264,2022-01-28,16:00,20.51,20.51,20.51,20.51,100,1
3,DE0005933931,EXS1,ISHS CORE DAX UC.ETF EOA,ETF,EUR,2504265,2022-01-28,16:00,129.54,129.54,129.50,129.50,220,2
4,LU0252634307,LYY8,MUL-LYX.DLY.LEVDAX U.E. A,ETF,EUR,2504268,2022-01-28,16:00,130.52,130.52,130.48,130.48,158,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12677,DE0007165631,SRT3,SARTORIUS AG VZO O.N.,Common stock,EUR,2505079,2022-01-28,16:44,465.50,465.50,465.50,465.50,10,1
12678,DE000A2N4H07,WEW,WESTWING GROUP INH. O.N.,Common stock,EUR,3490682,2022-01-28,16:44,20.62,20.62,20.62,20.62,1,1
12679,LU1899270539,ECBI,SI UC.-UC M.E.G.B.E. EOA,ETF,EUR,3629672,2022-01-28,16:52,107.50,107.50,107.50,107.50,48,1
12680,LU2249056297,MWSH,AIS-AM.MS.W SRI DRHEOA,ETF,EUR,5904190,2022-01-28,16:52,57.75,57.75,57.75,57.75,56,1


In [21]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']

-- only pulls in data under the specific columns

df_all = df_all.loc[:, columns]

-- quick clean up
df_all.dropna(inplace = True)
df_all.shape

# 2. Transformations & adding columns for analysis
-- Get Opening Price per ISIN and day
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date']).['StartPrice'].transform('first')

-- Get closing Price per ISIN and day
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date']).['StartPrice'].transform('last')

df_all[df_all['ISIN']== 'LU1899270539']

## Aggregations
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))


-- % change prev closing
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['previous_closing_price']) / df_all['previous_closing_price'] * 100

### write to s3

key = 'xetra_notes' + '.parquet'
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index-False)
bucket_target = s3.Bucket('xetra-12')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

## reading the uploaded file to check

for obj in bucket_target.objects.all()
    print(obj.key)
    
prq_obj = bucket_target.Object(key='xetra.parquet').get().get('Body').read()
dara = BytesIO(prq_obj)
df_report = pd.read_parquet(data)